<a href="https://colab.research.google.com/github/vpw/TSAI-END3/blob/main/Assignment6/Assignment6_Quora.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
import numpy as np

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [11]:
#!wget http://qim.fs.quoracdn.net/quora_duplicate_questions.tsv

In [12]:
qq_filename = "/content/quora_onlydup_questions.tsv"

In [13]:
SOS_token = 0
EOS_token = 1

class QQ:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [14]:
# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters

def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

## Input data format


![Fields image](https://qph.fs.quoracdn.net/main-qimg-ea50c7a005eb7750af0b53b07c8caa60)

        Fields:
        
        id - ID of the question pair data entry
        qid1, qid2 - unique IDs of the questions in the quora database
        question1, question 2 - content of the 2 questions per entry
        is_duplicate - whether the 2 questions can be considered duplicate in terms of their meaning

        Fields:
        
        id - ID of the question pair data entry
        qid1, qid2 - unique IDs of the questions in the quora database
        question1, question 2 - content of the 2 questions per entry
        is_duplicate - whether the 2 questions can be considered duplicate in terms of their meaning




In [15]:

def readQQs(q_name, dupq_name):
    print("Reading lines...")

    # Read the file and split into lines
    lines = open(qq_filename, encoding='utf-8', errors='ignore').\
        read().strip().split('\n')

    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')[3:5]] for l in lines[1:]]
#    print(pairs[0], pairs[0][0], pairs[0][1])
#    print(pairs[1], pairs[1][0], pairs[1][1])
#    print("pairs shape ",np.shape(pairs))

    questions = QQ(q_name)
    dupquestions = QQ(dupq_name)

    return questions, dupquestions, pairs

In [16]:
MAX_LENGTH=12

# could add other filters here like the easy/med/hard question and answer types
def filterPair(p):
    if len(p) < 2:
        return False
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH

def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [17]:
def prepareData(q_name, dupq_name):
    questions, dupquestions, pairs = readQQs(q_name, dupq_name)
    print(pairs[0])
    print("Read %s question pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s question pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
#        print(pair)
        questions.addSentence(pair[0])
        dupquestions.addSentence(pair[1])
    print("Counted words:")
    print(questions.name, questions.n_words)
    print(dupquestions.name, dupquestions.n_words)
    return questions, dupquestions, pairs

questions, dupquestions, pairs = prepareData('quora_qs', 'quora_dupqs')
#pairs.shape

Reading lines...
['astrology i am a capricorn sun cap moon and cap rising . . .what does that say about me ?', 'i m a triple capricorn sun moon and ascendant in capricorn what does this say about me ?']
Read 40908 question pairs
Trimmed to 22404 question pairs
Counting words...
Counted words:
quora_qs 10108
quora_dupqs 10096


In [18]:
for _ in range(5):
  print(random.choice(pairs))

['why does mickey mouse watch anime ?', 'why does minnie mouse watch anime ?']
['what is an abstract class ?', 'what is abstract class ?']
['what recommendations would you like to give to walt disney ?', 'recommendations to walt disney ?']
['how can you prove time dilation ?', 'is time dilation real ?']
['what makes a person a good problem solver ?', 'how can i be a fast problem solver ?']


In [19]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [20]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        
        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [21]:
def indexesFromSentence(qq, sentence):
    return [qq.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(qq, sentence):
    indexes = indexesFromSentence(qq, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(questions, pair[0])
    target_tensor = tensorFromSentence(dupquestions, pair[1])
    return (input_tensor, target_tensor)

In [22]:
teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [23]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))



In [24]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

In [25]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [30]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(questions, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(dupquestions.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

In [27]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [28]:
hidden_size = 256
encoder1 = EncoderRNN(questions.n_words, hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, dupquestions.n_words, dropout_p=0.1).to(device)

trainIters(encoder1, attn_decoder1, 75000, print_every=5000)

3m 53s (- 54m 34s) (5000 6%) 4.3584
7m 41s (- 50m 1s) (10000 13%) 3.8761
11m 29s (- 45m 59s) (15000 20%) 3.6009
15m 18s (- 42m 5s) (20000 26%) 3.4004
19m 9s (- 38m 18s) (25000 33%) 3.2962
22m 59s (- 34m 28s) (30000 40%) 3.1678
26m 49s (- 30m 38s) (35000 46%) 3.0587
30m 39s (- 26m 49s) (40000 53%) 2.9845
34m 29s (- 22m 59s) (45000 60%) 2.8974
38m 21s (- 19m 10s) (50000 66%) 2.8274
42m 12s (- 15m 20s) (55000 73%) 2.7859
46m 3s (- 11m 30s) (60000 80%) 2.7280
49m 55s (- 7m 40s) (65000 86%) 2.6517
53m 47s (- 3m 50s) (70000 93%) 2.6326
57m 39s (- 0m 0s) (75000 100%) 2.5869


In [32]:
evaluateRandomly(encoder1, attn_decoder1)

> should i have snapchat ?
= should i get snapchat ?
< should i i have ? <EOS>

> does green tea really reduces weight ?
= does green chai tea assist with weight loss ?
< does green tea burn belly fat ? <EOS>

> what is the fastest way to lose weight safely ?
= how can i lose weight quickly ?
< what is the fastest way to lose weight ? <EOS>

> what is the best gift you are ever received ?
= what is the best physical gift you ve ever received ?
< what is the best gift gift ? <EOS>

> is it physically possible to travel back in time ?
= is time travel possible after the discovery of gravitational waves ?
< is it possible time travel in travel ? <EOS>

> why does isis never disturb israel ?
= why isn t isis attacking israel ?
< why do isis israel israel ? <EOS>

> which is the best mobile below ?
= which is best mobile under ?
< which mobile best mobile best mobile ? <EOS>

> what is digital marketing exactly ?
= what are the basics of digital marketing ?
< what are the marketing on ? <EO